In [32]:
import pandas as pd
import numpy as np

from yahooquery import Ticker
from shared_code import utils

# Piotroski Score Based on Fundamentals

In [3]:
azure_utils = utils.AzureUtils()
secret_client = azure_utils.initialize_key_vault()

sa_secret = azure_utils.get_key_vault_secret(secret_client, 'sa-secret')
sa_name = azure_utils.get_key_vault_secret(secret_client, 'sa-name')

azure_utils.initialize_storage_account_ad(sa_secret.value, sa_name.value)
azure_utils.initialize_data_lake(sa_name.value, sa_secret.value)

In [3]:
dim_company = azure_utils.download_parquet_blob("bronze/companies/sp500", "companies_sp500.parquet")
dim_company.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [ ]:
# class StockFundamentals:
#     def __init__(self, symbol):
#         self.symbol = symbol
#         self.ticker = Ticker(self.symbol)
#         self.income_statement = None
#         self.balance_sheet = None
#         self.cash_flow = None

#     def fetch_data(self):
#         self.income_statement = self.ticker.income_statement(frequency = 'q', trailing = False).sort_values('asOfDate').set_index("asOfDate")
#         self.balance_sheet = self.ticker.balance_sheet(frequency = 'q', trailing = False).sort_values('asOfDate').set_index("asOfDate")
#         self.cash_flow = self.ticker.cash_flow(frequency = 'q', trailing = False).sort_values('asOfDate').set_index("asOfDate")

# class PiotroskiScoreCalculator:
#     def __init__(self, stock):
#         self.stock = stock
#         self.date = None
#         self.prev_date = None

#     def set_dates(self, date):
#         self.date = date
#         self.prev_date = self.stock.balance_sheet.index[self.stock.balance_sheet.index.get_loc(date) - 1]

#     def calculate_roa_score(self):
#         roa = self.stock.income_statement.loc[self.date, 'NetIncome'] / self.stock.balance_sheet.loc[self.date, 'TotalAssets']
#         return 1 if roa > 0 else 0

#     def calculate_cfo_score(self):
#         cfo = self.stock.cash_flow.loc[self.date, 'OperatingCashFlow']
#         return 1 if cfo > 0 else 0

#     def calculate_delta_roa_score(self):
#         roa = self.stock.income_statement.loc[self.date, 'NetIncome'] / self.stock.balance_sheet.loc[self.date, 'TotalAssets']
#         delta_roa = roa - (self.stock.income_statement.loc[self.prev_date, 'NetIncome'] / \
#                            self.stock.balance_sheet.loc[self.prev_date, 'TotalAssets'])
#         return 1 if delta_roa > 0 else 0

#     def calculate_quality_of_earnings_score(self):
#         cfo = self.stock.cash_flow.loc[self.date, 'OperatingCashFlow']
#         return 1 if cfo > self.stock.income_statement.loc[self.date, 'NetIncome'] else 0

#     def calculate_delta_leverage_score(self):
#         delta_leverage = self.stock.balance_sheet.loc[self.date, 'LongTermDebt'] / self.stock.balance_sheet.loc[self.date, 'TotalAssets'] - \
#                          self.stock.balance_sheet.loc[self.prev_date, 'LongTermDebt'] / self.stock.balance_sheet.loc[self.prev_date, 'TotalAssets']
#         return 1 if delta_leverage < 0 else 0

#     def calculate_delta_liquidity_score(self):
#         delta_liquidity = (self.stock.balance_sheet.loc[self.date, 'CurrentAssets'] / self.stock.balance_sheet.loc[self.date, 'CurrentLiabilities']) - \
#                           (self.stock.balance_sheet.loc[self.prev_date, 'CurrentAssets'] / self.stock.balance_sheet.loc[self.prev_date, 'CurrentLiabilities'])
#         return 1 if delta_liquidity > 0 else 0

#     def calculate_new_equity_score(self):
#         new_equity = self.stock.balance_sheet.loc[self.date, 'CommonStock'] - \
#                      self.stock.balance_sheet.loc[self.prev_date, 'CommonStock']
#         return 1 if new_equity <= 0 else 0

#     def calculate_gross_margin_score(self):
#         gross_margin_now = (self.stock.income_statement.loc[self.date, 'TotalRevenue'] - \
#                             self.stock.income_statement.loc[self.date, 'CostOfRevenue']) / self.stock.income_statement.loc[self.date, 'TotalRevenue']
#         gross_margin_prev = (self.stock.income_statement.loc[self.prev_date, 'TotalRevenue'] - \
#                              self.stock.income_statement.loc[self.prev_date, 'CostOfRevenue']) / self.stock.income_statement.loc[self.prev_date, 'TotalRevenue']
#         return 1 if gross_margin_now > gross_margin_prev else 0

#     def calculate_asset_turnover_score(self):
#         asset_turnover_now = self.stock.income_statement.loc[self.date, 'TotalRevenue'] / self.stock.balance_sheet.loc[self.date, 'TotalAssets']
#         asset_turnover_prev = self.stock.income_statement.loc[self.prev_date,
#         'TotalRevenue'] / self.stock.balance_sheet.loc[self.prev_date, 'TotalAssets']
#         return 1 if asset_turnover_now > asset_turnover_prev else 0

#     def calculate_score(self):
#         self.stock.fetch_data()
#         score_data = []

#         for date in self.stock.balance_sheet.index[1:]:
#             self.set_dates(date)

#             roa_score = self.calculate_roa_score()
#             cfo_score = self.calculate_cfo_score()
#             delta_roa_score = self.calculate_delta_roa_score()
#             quality_of_earnings_score = self.calculate_quality_of_earnings_score()
#             delta_leverage_score = self.calculate_delta_leverage_score()
#             delta_liquidity_score = self.calculate_delta_liquidity_score()
#             new_equity_score = self.calculate_new_equity_score()
#             gross_margin_score = self.calculate_gross_margin_score()
#             asset_turnover_score = self.calculate_asset_turnover_score()

#             total_score = sum([roa_score, cfo_score, delta_roa_score, quality_of_earnings_score, delta_leverage_score,
#                                 delta_liquidity_score, new_equity_score, gross_margin_score, asset_turnover_score])
            
#             # Save the data
#             score_data.append([date, roa_score, cfo_score, delta_roa_score, quality_of_earnings_score, 
#                                delta_leverage_score, delta_liquidity_score, new_equity_score, 
#                                gross_margin_score, asset_turnover_score, total_score])

#         # Convert the data into a DataFrame
#         scores_df = pd.DataFrame(score_data, columns=['Date', 'ROA', 'CFO', 'Delta ROA', 'Quality of Earnings', 
#                                                       'Delta Leverage', 'Delta Liquidity', 'New Equity', 
#                                                       'Gross Margin', 'Asset Turnover', 'Piotroski Score'])
#         scores_df.set_index('Date', inplace=True)

#         return scores_df


In [ ]:

# Use the classes
stocks = ['AAPL', 'MSFT', 'GOOGL']  # Add any stocks you're interested in
all_scores = []

for symbol in stocks:
    try:
        stock = utils.StockFundamentals(symbol)
        calculator = utils.PiotroskiScoreCalculator(stock)
        scores = calculator.calculate_score()
        scores['Ticker'] = symbol  # add a column for the ticker
        all_scores.append(scores)
    except Exception as e:
        print(f'Couldnt fetch data for {symbol} with error {e}')
        continue

# Concatenate all the individual DataFrames together
all_scores_df = pd.concat(all_scores)

all_scores_df

In [ ]:
scores_df = pd.DataFrame(scores)

# Get the last valid score and date for each ticker
latest_scores = {ticker: {"Date": scores_df[ticker].last_valid_index(), "Score": scores_df[ticker][scores_df[ticker].last_valid_index()]} for ticker in scores_df.columns}

# Convert to a DataFrame
latest_scores_df = pd.DataFrame(latest_scores).T

latest_scores_df


In [ ]:
# Only keep the 'GICS Sector' column
df_ticker_sector = dim_company[['Security', 'GICS Sector']]

# Join the two dataframes
merged_df = df_ticker_sector.join(latest_scores_df)

# Sort by 'GICS Sector' and 'Score'
merged_df.sort_values(by=['GICS Sector', 'Score'], ascending=[True, False], inplace=True)

merged_df


In [ ]:
# Only keep the 'GICS Sector' column
df_ticker_sector = dim_company[['Security', 'GICS Sub-Industry']]

# Join the two dataframes
merged_df = df_ticker_sector.join(latest_scores_df)

# Sort by 'GICS Sector' and 'Score'
merged_df.sort_values(by=['GICS Sub-Industry', 'Score'], ascending=[True, False], inplace=True)

merged_df


# Company Info

In [37]:
MSFT = Ticker("MSFT")
df_profile = pd.DataFrame(MSFT.asset_profile).T

In [42]:
df_profile = df_profile.drop(columns = ["companyOfficers"])

In [43]:
df_profile

,address1,auditRisk,boardRisk,city,compensationAsOfEpochDate,compensationRisk,country,fax,fullTimeEmployees,governanceEpochDate,...,industryDisp,longBusinessSummary,maxAge,overallRisk,phone,sector,shareHolderRightsRisk,state,website,zip
MSFT,One Microsoft Way,6,3,Redmond,2022-12-31 01:00:00,3,United States,425 706 7329,221000,2023-06-01 02:00:00,...,Software—Infrastructure,"Microsoft Corporation develops, licenses, and ...",86400,2,425 882 8080,Technology,2,WA,https://www.microsoft.com,98052-6399


In [46]:
df_AssetProfile = azure_utils.ingest_bronze_data(f"AssetProfile/")
df_AssetProfile = df_AssetProfile.rename(columns =
    {
        "index": "Ticker"
    }
)
df_AssetProfile

,Ticker,address1,auditRisk,boardRisk,city,compensationAsOfEpochDate,compensationRisk,country,fax,fullTimeEmployees,...,industryDisp,longBusinessSummary,maxAge,overallRisk,phone,sector,shareHolderRightsRisk,website,zip,state
0,AAK.ST,Skrivaregatan 9,5,4,Malmö,2022-12-31 00:00:00,9,Sweden,46 4 06 27 83 11,4000,...,Packaged Foods,AAK AB (publ.) develops and sells plant-based ...,86400,7,46 4 06 27 83 00,Consumer Defensive,1,https://www.aak.com,215 32,NaN
1,MSFT,One Microsoft Way,6,3,Redmond,2022-12-31 00:00:00,3,United States,425 706 7329,221000,...,Software—Infrastructure,"Microsoft Corporation develops, licenses, and ...",86400,2,425 882 8080,Technology,2,https://www.microsoft.com,98052-6399,WA


In [47]:
parquet_file = df_AssetProfile.to_parquet(index = False)
azure_utils.upload_blob(parquet_file, "gold/factassetprofile", "fact_assetProfile.parquet")

In [87]:
from datetime import date

df_financialData = pd.DataFrame(MSFT.financial_data).T
df_priceTargets = df_financialData[["currentPrice", "numberOfAnalystOpinions", "recommendationKey", "recommendationMean", "targetLowPrice", "targetMeanPrice", "targetMedianPrice"]].reset_index()
df_priceTargets = df_priceTargets.rename(columns =
        {
            "index": "Ticker"
        }
    )
df_priceTargets["observationDate"] = date.today()
df_priceTargets

,Ticker,currentPrice,numberOfAnalystOpinions,recommendationKey,recommendationMean,targetLowPrice,targetMeanPrice,targetMedianPrice,observationDate
0,MSFT,323.38,43,buy,1.8,232.0,340.28,340.0,2023-06-07


In [88]:
parquet_file = df_priceTargets.to_parquet(index = False)
azure_utils.upload_blob(parquet_file, "gold/factpricetarget", "fact_priceTarget.parquet")

In [64]:
azure_utils.download_parquet_blob(f"gold/factpricetarget", f"fact_priceTarget.parquet")

,Ticker,currentPrice,numberOfAnalystOpinions,recommendationKey,recommendationMean,targetLowPrice,targetMeanPrice,targetMedianPrice,observationDate
0,MSFT,323.38,43,buy,1.8,232.0,340.28,340.0,2023-06-07


In [67]:
df_priceTargets.drop_duplicates()

,Ticker,currentPrice,numberOfAnalystOpinions,recommendationKey,recommendationMean,targetLowPrice,targetMeanPrice,targetMedianPrice,observationDate
0,MSFT,323.38,43,buy,1.8,232.0,340.28,340.0,2023-06-07


In [68]:
azure_utils.ingest_bronze_data(f"FinancialData/")

,index,currentPrice,currentRatio,debtToEquity,earningsGrowth,ebitdaMargins,financialCurrency,grossMargins,maxAge,operatingMargins,...,recommendationMean,revenuePerShare,targetHighPrice,targetLowPrice,targetMeanPrice,targetMedianPrice,totalCash,totalCashPerShare,totalDebt,totalRevenue
0,ARP.ST,62.00,2.451,9.854,-0.103,0.19243,PLN,0.28645,86400,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AZN.ST,1591.50,0.896,90.676,3.652,0.33955,USD,0.84304,86400,0.23098,...,1.9,28.316,1716.32,889.28,1418.30,1649.29,6.462000e+09,4.172,3.247200e+10,4.384000e+10
2,BACTI-B.ST,77.60,3.534,51.139,NaN,-0.10770,SEK,0.79096,86400,-0.22602,...,4.0,6.640,NaN,NaN,NaN,NaN,1.866480e+08,5.326,2.420040e+08,2.326780e+08
3,BALCO.ST,48.10,1.374,27.215,-0.268,0.08655,SEK,0.21000,86400,0.07249,...,3.0,62.570,46.00,46.00,46.00,46.00,1.640000e+07,0.749,2.030000e+08,1.370859e+09
4,BALD-B.ST,42.45,1.002,140.323,NaN,0.69245,SEK,0.68898,86400,0.68898,...,2.0,10.982,65.00,45.00,55.83,57.50,6.572000e+09,5.695,1.433020e+11,1.238500e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,XYL,107.08,2.015,54.277,0.200,0.13987,USD,0.38031,86400,0.09810,...,2.4,31.613,135.00,105.00,121.50,123.00,8.370000e+08,4.634,1.948000e+09,5.698000e+09
629,YUM,132.44,0.994,NaN,-0.228,0.33775,USD,0.48501,86400,0.31787,...,2.3,24.437,172.00,132.00,148.47,150.00,3.730000e+08,1.332,1.249100e+10,6.940000e+09
630,ZBH,133.07,1.889,49.191,15.353,0.32870,USD,0.71576,86400,0.19864,...,2.7,33.903,172.00,116.00,150.16,152.00,3.302000e+08,1.583,5.980400e+09,7.107700e+09
631,ZBRA,277.56,0.877,79.142,-0.243,0.20003,USD,0.46124,86400,0.16597,...,2.2,111.065,410.00,251.00,330.50,335.00,1.090000e+08,2.119,2.288000e+09,5.754000e+09


In [90]:
fact_priceTarget = azure_utils.download_parquet_blob(f"gold/factpricetarget", f"fact_priceTarget.parquet").drop_duplicates()

df_FinancialData = azure_utils.ingest_bronze_data(f"FinancialData/").drop_duplicates()
df_priceTargets = df_FinancialData[["currentPrice", "numberOfAnalystOpinions", "recommendationKey", "recommendationMean", "targetLowPrice", "targetMeanPrice", "targetMedianPrice"]]
df_priceTargets = df_priceTargets.rename(columns =
        {
            "index": "Ticker"
        }
    )
df_priceTargets["observationDate"] = date.today()



In [92]:
df_priceTargets

,currentPrice,numberOfAnalystOpinions,recommendationKey,recommendationMean,targetLowPrice,targetMeanPrice,targetMedianPrice,observationDate
0,116.59,16.0,buy,2.0,126.0,145.81,144.5,2023-06-07
1,210.40,10.0,buy,2.2,161.0,231.00,235.0,2023-06-07
2,15.16,13.0,hold,2.9,9.5,17.05,17.0,2023-06-07
3,65.54,18.0,hold,2.9,57.0,82.54,80.0,2023-06-07
4,177.82,38.0,buy,2.0,140.0,185.74,186.5,2023-06-07
...,...,...,...,...,...,...,...,...
721,132.44,19.0,buy,2.3,132.0,148.47,150.0,2023-06-07
722,133.07,22.0,hold,2.7,116.0,150.16,152.0,2023-06-07
723,277.56,12.0,buy,2.2,251.0,330.50,335.0,2023-06-07
724,163.58,12.0,buy,1.7,180.0,215.08,207.0,2023-06-07


In [91]:
pd.concat([df_priceTargets[~df_priceTargets.index.isin(fact_priceTarget.index)], fact_priceTarget]).reset_index()

,index,currentPrice,numberOfAnalystOpinions,recommendationKey,recommendationMean,targetLowPrice,targetMeanPrice,targetMedianPrice,observationDate,Ticker
0,1,210.40,10.0,buy,2.2,161.00,231.00,235.00,2023-06-07,NaN
1,2,15.16,13.0,hold,2.9,9.50,17.05,17.00,2023-06-07,NaN
2,3,65.54,18.0,hold,2.9,57.00,82.54,80.00,2023-06-07,NaN
3,4,177.82,38.0,buy,2.0,140.00,185.74,186.50,2023-06-07,NaN
4,5,408.00,3.0,buy,2.5,384.89,409.83,414.82,2023-06-07,NaN
...,...,...,...,...,...,...,...,...,...,...
721,722,133.07,22.0,hold,2.7,116.00,150.16,152.00,2023-06-07,NaN
722,723,277.56,12.0,buy,2.2,251.00,330.50,335.00,2023-06-07,NaN
723,724,163.58,12.0,buy,1.7,180.00,215.08,207.00,2023-06-07,NaN
724,800,133.33,11.0,buy,2.5,130.00,148.64,147.00,2023-06-07,NaN


In [79]:
pd.concat([df_priceTargets[~df_priceTargets.index.isin(fact_priceTarget.index)], fact_priceTarget])

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'

In [84]:
df_priceTargets

,,currentPrice,numberOfAnalystOpinions,recommendationKey,recommendationMean,targetLowPrice,targetMeanPrice,targetMedianPrice
Ticker,observationDate,,,,,,,
0,2023-06-07,116.59,16.0,buy,2.0,126.0,145.81,144.5
1,2023-06-07,210.40,10.0,buy,2.2,161.0,231.00,235.0
2,2023-06-07,15.16,13.0,hold,2.9,9.5,17.05,17.0
3,2023-06-07,65.54,18.0,hold,2.9,57.0,82.54,80.0
4,2023-06-07,177.82,38.0,buy,2.0,140.0,185.74,186.5
...,...,...,...,...,...,...,...,...
718,2023-06-07,163.58,12.0,buy,1.7,180.0,215.08,207.0
737,2023-06-07,204.15,14.0,buy,1.9,172.0,226.29,231.5
759,2023-06-07,117.14,5.0,buy,2.3,120.0,127.80,125.0
